In [1]:
from transformers import AutoTokenizer
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../Data/Train_data/BGE_train.csv")

In [3]:
ctxs = df["oracle_context"].to_list()
qsns = df['question'].to_list()

In [4]:
batches = [ctxs[i:i+1024] for i in range(0, len(ctxs), 1024)]

In [5]:
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-large-en-v1.5")

In [6]:
oracle_enc = []
for batch in batches:
    encs = tokenizer.batch_encode_plus(batch)
    for i in range(len(batch)):
        oracle_enc.append(encs[i])

Token indices sequence length is longer than the specified maximum sequence length for this model (1982 > 512). Running this sequence through the model will result in indexing errors


In [7]:
new_ids = []

In [8]:
def split_encs(arr, chunk_size=512):
    """
    Splits a NumPy array into chunks of at most `chunk_size` elements.

    Parameters:
    arr (np.ndarray): The input array to split.
    chunk_size (int): The maximum size of each chunk.

    Returns:
    list of np.ndarray: A list of NumPy arrays, each with at most `chunk_size` elements.
    """
    # Calculate the number of chunks needed
    num_chunks = int(np.ceil(len(arr) / chunk_size))
    return [arr[i*chunk_size:(i+1)*chunk_size] for i in range(num_chunks)]

In [9]:
for i in range(len(ctxs)):
  new_ids.append(split_encs(np.array(oracle_enc[i].ids[1:-1])))

In [10]:
new_ctxs = []

In [11]:
for ids in new_ids:
  new_ctxs.append(tokenizer.batch_decode(ids))

In [12]:
question = []
oracles = []

In [13]:
for q,e in zip(qsns,new_ctxs):
  for c in e:
    question.append(q)
    oracles.append(c)

In [14]:
df_new  = pd.DataFrame({"question" : question, 'oracle_context' : oracles})

In [15]:
batches = [oracles[i:i+1024] for i in range(0, len(oracles), 1024)]

In [16]:
new_oracle_enc = []
for batch in batches:
    encs = tokenizer.batch_encode_plus(batch)
    for i in range(len(batch)):
        new_oracle_enc.append(encs[i])

In [17]:
new_oracle_lens = []
for i in range(len(new_ctxs)):
    new_oracle_lens.append(len(new_oracle_enc[i].ids))

In [18]:
new_oracle_lens = np.array(new_oracle_lens)

In [19]:
new_oracle_lens.max()

1157

In [21]:
idxs = np.where(new_oracle_lens < 100)[0]

In [22]:
final_df = df_new.drop(idxs)
final_df.reset_index(drop=True, inplace=True)

In [23]:
final_df

,question,oracle_context
0,What happens if a thread requests a resource t...,the thread requests the resource. if the reque...
1,What happens if a thread requests a resource t...,represented by chopsticks. if all the philosop...
2,What happens if a thread requests a resource t...,( void * param ) _ { _ pthread mutex lock ( & ...
3,What happens if a thread requests a resource t...,. the code example in figure 8. 2 rewrites the...
4,How can a thread operate on a resource in a mu...,the thread requests the resource. if the reque...
...,...,...
12819,Why is the SETUID bit commonly used for sensit...,for a data file the owner wants to make public...
12820,Why is the SETUID bit commonly used for sensit...,users do have a legitimate need to print somet...
12821,How can access to I/O devices be controlled in...,"for simplicity, we will not discuss this featu..."
12822,How can access to I/O devices be controlled in...,for a data file the owner wants to make public...


In [24]:
final_df.to_csv("../Data/Train_data/BGE-finetune-data.csv",index=False)